In [2]:
import os
import numpy as np
import pandas as pd
from numba import jit
from tqdm import tqdm
import statsmodels.api as sm
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import Alpha as alpha

import warnings
warnings.filterwarnings('ignore')

# CDVDV_V

In [5]:
def cdpdv_func(date):
    close_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\close",date)
    volume_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\volume",date)
    close = pd.read_parquet(close_path)
    volume = pd.read_parquet(volume_path)
    
    code = close.columns
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    close = close.values
    volume = volume.values
    
    dv_t = np.diff(volume,axis=0)[:-1]
    dp_t_1 = np.diff(close,axis=0)[1:]
    
    # dpdv_pos
    dv_t_pos = np.where(np.logical_and(dv_t > 0, dp_t_1 > 0), dv_t, np.nan)
    dp_t_1_pos = np.where(np.logical_and(dv_t > 0, dp_t_1 > 0), dp_t_1, np.nan)
    cov = np.nanmean((dv_t_pos-np.nanmean(dv_t_pos,axis=0))*(dp_t_1_pos-np.nanmean(dp_t_1_pos,axis=0)),axis=0)
    dpdv_pos = cov/np.nanstd(dv_t_pos,axis=0)/np.nanstd(dp_t_1_pos,axis=0)

    # dpdv_neg
    dv_t_neg = np.where(np.logical_and(dv_t < 0, dp_t_1 < 0), dv_t, np.nan)
    dp_t_1_neg = np.where(np.logical_and(dv_t < 0, dp_t_1 < 0), dp_t_1, np.nan)
    cov = np.nanmean((dv_t_neg-np.nanmean(dv_t_neg,axis=0))*(dp_t_1_neg-np.nanmean(dp_t_1_neg,axis=0)),axis=0)
    dpdv_neg = cov/np.nanstd(dv_t_neg,axis=0)/np.nanstd(dp_t_1_neg,axis=0)

    # dpdv_pos_neg
    dv_t_pos = np.where(np.logical_and(dv_t > 0, dp_t_1 < 0), dv_t, np.nan)
    dp_t_1_neg = np.where(np.logical_and(dv_t > 0, dp_t_1 < 0), dp_t_1, np.nan)
    cov = np.nanmean((dv_t_pos-np.nanmean(dv_t_pos,axis=0))*(dp_t_1_neg-np.nanmean(dp_t_1_neg,axis=0)),axis=0)
    dpdv_pos_neg = cov/np.nanstd(dv_t_pos,axis=0)/np.nanstd(dp_t_1_neg,axis=0)

    # dpdv_neg_pos
    dp_t_neg = np.where(np.logical_and(dv_t < 0, dp_t_1 > 0), dv_t, np.nan)
    dp_t_1_pos = np.where(np.logical_and(dv_t < 0, dp_t_1 > 0), dp_t_1, np.nan)
    cov = np.nanmean((dp_t_neg-np.nanmean(dp_t_neg,axis=0))*(dp_t_1_pos-np.nanmean(dp_t_1_pos,axis=0)),axis=0)
    dpdv_neg_pos = cov/np.nanstd(dp_t_neg,axis=0)/np.nanstd(dp_t_1_pos,axis=0)
    
    dpdv_pos = pd.Series(dpdv_pos,index=code, name=date)
    dpdv_neg = pd.Series(dpdv_neg,index=code, name=date)
    dpdv_pos_neg = pd.Series(dpdv_pos_neg,index=code, name=date)
    dpdv_neg_pos = pd.Series(dpdv_neg_pos,index=code, name=date)
    
    return dpdv_pos, dpdv_neg, dpdv_pos_neg, dpdv_neg_pos
    
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\close")

DPDV_POS = []
DPDV_NEG = []
DPDV_POS_NEG = []
DPDV_NEG_POS = []

for i in tqdm(data):
    
    dpdv_pos, dpdv_neg, dpdv_pos_neg, dpdv_neg_pos = cdpdv_func(date=i)
    DPDV_POS.append(dpdv_pos)
    DPDV_NEG.append(dpdv_neg)
    DPDV_POS_NEG.append(dpdv_pos_neg)
    DPDV_NEG_POS.append(dpdv_neg_pos)
    
DPDV_POS = pd.concat(DPDV_POS, axis=1).T
DPDV_NEG = pd.concat(DPDV_NEG, axis=1).T
    
DPDV_POS_NEG = pd.concat(DPDV_POS_NEG, axis=1).T
DPDV_NEG_POS = pd.concat(DPDV_NEG_POS, axis=1).T

100%|██████████████████████████████████████████████████████████████████████████████| 2512/2512 [24:32<00:00,  1.71it/s]


In [6]:
dv_pos_dp_pos_corr = alpha.mvind_neutral(DPDV_POS.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])
dv_neg_dp_neg_corr =alpha.mvind_neutral(DPDV_NEG.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])
dv_pos_dp_neg_corr = alpha.mvind_neutral(DPDV_POS_NEG.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])
dv_neg_dp_pos_corr =alpha.mvind_neutral(DPDV_NEG_POS.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:].loc[:'2024-03'])

CDPDV_V = alpha.zscore(dv_pos_dp_pos_corr)+alpha.zscore(dv_neg_dp_neg_corr)-alpha.zscore(dv_pos_dp_neg_corr)-alpha.zscore(dv_neg_dp_pos_corr)

alpha.to_fctDataBase(CDPDV_V.dropna(axis=0,how='all'),'CDPDV_V')

100%|██████████████████████████████████████████████████████████████████████████████| 2473/2473 [00:38<00:00, 65.06it/s]
